In [2]:
import numpy as np
import matplotlib.pyplot as plt

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [3]:
#bird img
import matplotlib.image as mpimg
import pandas as pd
import os
import torch
import torchaudio
from torchaudio import transforms
from torchaudio.utils import download_asset
import random
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
class imgdata(Dataset):
    def __init__(self, df, data_path, transform = None):
        self.df = df
        self.data_path = str(data_path)
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        # Absolute file path of the audio file - concatenate the audio directory with
        # the relative path
        img_file = self.data_path + self.df.loc[idx, 'Path']
        # Get the Class ID
        class_id = self.df.loc[idx, 'ID']
        img = mpimg.imread(img_file)
        if self.transform is not None:
            img = self.transform(img)
        return img, class_id
df_train = pd.read_csv("C:/Users/carlv/Downloads/trainfiles.csv")
df_valid = pd.read_csv("C:/Users/carlv/Downloads/Validationfiles.csv")
birdIMG_path = "C:/Users/carlv/Downloads/archive/"


train_transform = transforms.Compose([transforms.ToPILImage(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean,std)])
valid_transform = transforms.Compose([transforms.ToPILImage(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean,std)])
trainingDS = imgdata(df_train, birdIMG_path, train_transform)
train_dl = torch.utils.data.DataLoader(trainingDS, batch_size=25, shuffle=True)
validDS = imgdata(df_valid,birdIMG_path, train_transform )
valid_dl = torch.utils.data.DataLoader(validDS, batch_size=25, shuffle=False)
#img_data = (mpimg.imread("C:/Users/carlv/Downloads/archive/train/ABBOTTS BABBLER/001.jpg"))

In [4]:
trainimages, trainlabels = next(iter(train_dl))

In [5]:
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class imgClassifier (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(3, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Second Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Second Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=450)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)

        # Final output
        return x

myModel = imgClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

device(type='cpu')

In [6]:
def training(model, train_dl, num_epochs):
  # Loss Function, Optimizer and Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  # Repeat for each epoch
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0

    # Repeat for each batch in the training set
        for i, data in enumerate(train_dl):
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)
            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Keep stats for Loss and Accuracy
            running_loss += loss.item()

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs,1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

            #if i % 10 == 0:    # print every 10 mini-batches
            #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
    
        # Print stats at the end of the epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        acc = correct_prediction/total_prediction
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

    print('Finished Training')

num_epochs = 70   # adjustments needed
training(myModel, train_dl, num_epochs)

tensor([[ 0.0986,  0.0542, -0.0100,  ..., -0.0784, -0.1230, -0.3262],
        [-0.0529,  0.1958, -0.0269,  ...,  0.0808, -0.0100, -0.4183],
        [ 0.0392,  0.2316, -0.0136,  ..., -0.0351, -0.2401, -0.4297],
        ...,
        [-0.0194, -0.0498, -0.0350,  ...,  0.0422,  0.0720, -0.0233],
        [ 0.1357, -0.1084, -0.0393,  ..., -0.0738, -0.1184, -0.0436],
        [ 0.0796, -0.1881,  0.1396,  ..., -0.0936, -0.2618,  0.0036]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.0097,  0.1232,  0.1812,  ..., -0.0142, -0.2647, -0.3088],
        [ 0.0294,  0.0078, -0.0145,  ..., -0.1264, -0.0525, -0.3059],
        [-0.0702, -0.0733, -0.0844,  ...,  0.0282,  0.0376, -0.0415],
        ...,
        [-0.3204,  0.1386, -0.0898,  ...,  0.2280,  0.3266, -0.2187],
        [-0.1310, -0.4011, -0.0881,  ...,  0.1945,  0.2321,  0.1959],
        [-0.0092, -0.1070,  0.2261,  ..., -0.2664, -0.2952, -0.0843]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.0902, -0.1567, -0.1384,  ...,  0.0224,  0.1644,  0.012

KeyboardInterrupt: 